# Trabalho 3 - Iluminação

O objetivo do trabalho é construir um cenário 3D a partir de modelos/malhas pré-existentes e com aplicação de textura. O programa deve permitir explorar o cenário por meio de manipulação do pipeline de Viewing 3D (Matrizes Model x View x Projection).

No trabalho, será possível explorar uma casa simples, que possui em seu interior uma cama, um fogão e uma mesa. No exterior há um cacto e um poço, além de um gato passeando ao redor da casa. É um dia claro, sem nuvens no céu.

* a tecla P ativa e desativa o modo de visualização de malha poligonal;
* explore o cenário através do mouse e das teclas W, A, S, D do teclado;
* desativou-se o ponteiro do mouse para melhor experiência ao mover a câmera, então, para fechar a janela, pressione Alt+F4.


### Feito por:
* Caio Augusto Duarte Basso - NUSP 10801173
* Gabriel Garcia Lorencetti - NUSP 10691891

Observação: para a construção dos cenários e outros objetivos do trabalho, utilizou-se o código base disponibilizado pelo professor Ricardo Marcondes Marcacini durante a disciplina.

### Importando as bibliotecas necessárias.

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image

### Inicializando janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
largura = 1280
altura = 720
window = glfw.create_window(largura, altura, "Trabalho 3 - Iluminação", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec3.
* Definindo matrizes Model, View e Projection que acumulam transformações geométricas 3D e permitem navegação no cenário.
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base nas transformações Model, View e Projection.

In [3]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        attribute vec3 normals;
        
       
        varying vec2 out_texture;
        varying vec3 out_fragPos;
        varying vec3 out_normal;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
            out_fragPos = vec3(  model * vec4(position, 1.0));
            out_normal = vec3( model *vec4(normals, 1.0));            
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [4]:
fragment_code = """

        // parametro com a cor da(s) fonte(s) de iluminacao
        uniform vec3 lightPos; // define coordenadas de posicao da luz
        vec3 lightColor = vec3(1.0, 1.0, 1.0);
        
        // parametros da iluminacao ambiente e difusa
        uniform float ka; // coeficiente de reflexao ambiente
        uniform float kd; // coeficiente de reflexao difusa
        
        // parametros da iluminacao especular
        uniform vec3 viewPos; // define coordenadas com a posicao da camera/observador
        uniform float ks; // coeficiente de reflexao especular
        uniform float ns; // expoente de reflexao especular
        


        // parametros recebidos do vertex shader
        varying vec2 out_texture; // recebido do vertex shader
        varying vec3 out_normal; // recebido do vertex shader
        varying vec3 out_fragPos; // recebido do vertex shader
        uniform sampler2D samplerTexture;
        
        
        
        void main(){
        
            // calculando reflexao ambiente
            vec3 ambient = ka * lightColor;             
        
            // calculando reflexao difusa
            vec3 norm = normalize(out_normal); // normaliza vetores perpendiculares
            vec3 lightDir = normalize(lightPos - out_fragPos); // direcao da luz
            float diff = max(dot(norm, lightDir), 0.0); // verifica limite angular (entre 0 e 90)
            vec3 diffuse = kd * diff * lightColor; // iluminacao difusa
            
            // calculando reflexao especular
            vec3 viewDir = normalize(viewPos - out_fragPos); // direcao do observador/camera
            vec3 reflectDir = normalize(reflect(-lightDir, norm)); // direcao da reflexao
            float spec = pow(max(dot(viewDir, reflectDir), 0.0), ns);
            vec3 specular = ks * spec * lightColor;             
            
            // aplicando o modelo de iluminacao
            vec4 texture = texture2D(samplerTexture, out_texture);
            vec4 result = vec4((ambient + diffuse + specular),1.0) * texture; // aplica iluminacao
            gl_FragColor = result;

        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [5]:
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

### Associando nosso código-fonte aos slots solicitados

In [6]:
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [7]:
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [8]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [9]:
glAttachShader(program, vertex)
glAttachShader(program, fragment)

### Linkagem do programa

In [10]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Carregando Modelos (vértices e texturas) a partir de Arquivos

A função abaixo carrega modelos a partir de arquivos no formato WaveFront.


Para saber mais sobre o modelo, acesse: https://en.wikipedia.org/wiki/Wavefront_.obj_file


Nos slides e vídeo-aula da Aula 11 - Parte 1, nós descrevemos o funcionamento desse formato.

In [11]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    normals = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])

        ### recuperando vertices
        if values[0] == 'vn':
            normals.append(values[1:4])

        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat', 'o '):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            face_normals = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                face_normals.append(int(w[2]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, face_normals, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces
    model['normals'] = normals

    return model

In [12]:
glEnable(GL_TEXTURE_2D)
qtd_texturas = 10
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

### A lista abaixo armazena todos os vertices carregados dos arquivos

In [13]:
vertices_list = []    
normals_list = []    
textures_coord_list = []

### Vamos carregar cada modelo e definir funções para desenhá-los

#### Terreno**

In [14]:
modelo = load_model_from_file('cenario/terreno/terreno.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo terreno.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo terreno.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(1,'cenario/terreno/grass.png')


Processando modelo terreno.obj. Vertice inicial: 0
Processando modelo terreno.obj. Vertice final: 6


#### Casa

In [15]:
modelo = load_model_from_file('cenario/aldeia/aldeia.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo aldeia.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo aldeia.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(2,'cenario/aldeia/aldeia.png')

Processando modelo aldeia.obj. Vertice inicial: 6
Processando modelo aldeia.obj. Vertice final: 2592


#### Cama

In [16]:
modelo = load_model_from_file('cenario/cama/cama.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cama.obj. Vertice inicial:',len(vertices_list))
faces_visited = []
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo cama.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(3,'cenario/cama/metal_tile.jpg')
load_texture_from_file(4,'cenario/cama/cloth.jpg')
load_texture_from_file(5,'cenario/cama/cloth_2.jpg')


Processando modelo cama.obj. Vertice inicial: 2592
[1, 2, 3]  vertice inicial = 2592
[4, 5, 6]  vertice inicial = 2595
[7, 8, 9]  vertice inicial = 2598
[10, 11, 12]  vertice inicial = 2601
[12, 13, 14]  vertice inicial = 2604
[15, 16, 17]  vertice inicial = 2607
[18, 19, 20]  vertice inicial = 2610
[8, 21, 9]  vertice inicial = 2613
[22, 9, 5]  vertice inicial = 2616
[11, 3, 12]  vertice inicial = 2619
[5, 23, 6]  vertice inicial = 2622
[17, 24, 25]  vertice inicial = 2625
[26, 20, 27]  vertice inicial = 2628
[28, 21, 9]  vertice inicial = 2631
[29, 10, 8]  vertice inicial = 2634
[2, 4, 6]  vertice inicial = 2637
[19, 30, 31]  vertice inicial = 2640
[32, 33, 16]  vertice inicial = 2643
[34, 30, 35]  vertice inicial = 2646
[16, 36, 24]  vertice inicial = 2649
[15, 27, 32]  vertice inicial = 2652
[37, 35, 38]  vertice inicial = 2655
[39, 35, 19]  vertice inicial = 2658
[25, 34, 37]  vertice inicial = 2661
[40, 37, 11]  vertice inicial = 2664
[41, 38, 39]  vertice inicial = 2667
[11, 38,

[128, 152, 166]  vertice inicial = 3447
[126, 125, 150]  vertice inicial = 3450
[127, 134, 125]  vertice inicial = 3453
[203, 210, 121]  vertice inicial = 3456
[211, 217, 167]  vertice inicial = 3459
[205, 136, 212]  vertice inicial = 3462
[208, 121, 217]  vertice inicial = 3465
[161, 156, 157]  vertice inicial = 3468
[148, 143, 156]  vertice inicial = 3471
[218, 143, 144]  vertice inicial = 3477
[151, 150, 152]  vertice inicial = 3480
[128, 219, 130]  vertice inicial = 3489
[220, 195, 153]  vertice inicial = 3501
[146, 220, 154]  vertice inicial = 3504
[221, 141, 218]  vertice inicial = 3507
[147, 157, 220]  vertice inicial = 3510
[160, 159, 221]  vertice inicial = 3513
[195, 92, 153]  vertice inicial = 3516
[79, 93, 165]  vertice inicial = 3519
[164, 92, 86]  vertice inicial = 3522
[163, 90, 195]  vertice inicial = 3525
[156, 155, 163]  vertice inicial = 3528
[157, 163, 195]  vertice inicial = 3531
[154, 153, 164]  vertice inicial = 3534
[162, 89, 79]  vertice inicial = 3540
[165, 91

[13, 325, 323]  vertice inicial = 5877
[204, 326, 324]  vertice inicial = 5880
[109, 324, 325]  vertice inicial = 5883
[107, 321, 326]  vertice inicial = 5889
[21, 100, 327]  vertice inicial = 5895
[204, 107, 321]  vertice inicial = 5898
[109, 23, 326]  vertice inicial = 5904
[14, 323, 328]  vertice inicial = 5910
[2, 109, 13]  vertice inicial = 5916
[329, 3, 13]  vertice inicial = 5925
[2, 6, 204]  vertice inicial = 5946
[329, 14, 99]  vertice inicial = 5955
[91, 320, 327]  vertice inicial = 5964
[321, 320, 91]  vertice inicial = 5967
[324, 326, 91]  vertice inicial = 5970
[91, 322, 323]  vertice inicial = 5973
[91, 325, 324]  vertice inicial = 5979
[327, 322, 91]  vertice inicial = 5985
[91, 326, 321]  vertice inicial = 5988
[323, 325, 91]  vertice inicial = 5994
[327, 328, 91]  vertice inicial = 6006
[99, 322, 327]  vertice inicial = 6018
[100, 99, 328]  vertice inicial = 6030
[329, 10, 90]  vertice inicial = 6060
[6, 89, 182]  vertice inicial = 6063
[3, 168, 177]  vertice inicial =

#### Fogão

In [17]:
modelo = load_model_from_file('cenario/fogao/fogao.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo fogao.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo fogao.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(6,'cenario/fogao/fogao.jpg')

Processando modelo fogao.obj. Vertice inicial: 6600
Processando modelo fogao.obj. Vertice final: 215709


#### Gato

In [18]:
modelo = load_model_from_file('cenario/gato/gato.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo gato.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo gato.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(7,'cenario/gato/gato.jpg')

Processando modelo gato.obj. Vertice inicial: 215709
Processando modelo gato.obj. Vertice final: 427437


#### Cacto

In [19]:
modelo = load_model_from_file('cenario/cacto/cacto.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cacto.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo cacto.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(8,'cenario/cacto/cacto.jpg')

Processando modelo cacto.obj. Vertice inicial: 427437
Processando modelo cacto.obj. Vertice final: 482445


#### Poço

In [20]:
modelo = load_model_from_file('cenario/poco/poco.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo poco.obj. Vertice inicial:',len(vertices_list))
faces_visited = []
for face in modelo['faces']:
    if face[2] not in faces_visited:
        print(face[2], ' vertice inicial =', len(vertices_list))
        faces_visited.append(face[2])
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo poco.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(13,'cenario/poco/well_buket.tga')
load_texture_from_file(10,'cenario/poco/well_metal.tga')
load_texture_from_file(11,'cenario/poco/well_Roof.tga')
load_texture_from_file(14,'cenario/poco/Well_rope.tga')
load_texture_from_file(9,'cenario/poco/Well_Stone.tga')
load_texture_from_file(12,'cenario/poco/well_wood1.tga')


Processando modelo poco.obj. Vertice inicial: 482445
[1, 2, 3]  vertice inicial = 482445
[4, 5, 1]  vertice inicial = 482448
[6, 7, 4]  vertice inicial = 482451
[8, 9, 6]  vertice inicial = 482454
[10, 11, 12]  vertice inicial = 482457
[13, 14, 10]  vertice inicial = 482460
[15, 16, 13]  vertice inicial = 482463
[17, 18, 15]  vertice inicial = 482466
[19, 20, 17]  vertice inicial = 482469
[21, 22, 19]  vertice inicial = 482472
[23, 24, 21]  vertice inicial = 482475
[25, 26, 23]  vertice inicial = 482478
[27, 28, 29]  vertice inicial = 482481
[30, 31, 27]  vertice inicial = 482484
[32, 33, 30]  vertice inicial = 482487
[3, 34, 32]  vertice inicial = 482490
[30, 29, 3]  vertice inicial = 482493
[25, 19, 29]  vertice inicial = 482496
[23, 21, 19]  vertice inicial = 482499
[17, 10, 19]  vertice inicial = 482502
[15, 13, 10]  vertice inicial = 482505
[10, 8, 4]  vertice inicial = 482508
[8, 6, 4]  vertice inicial = 482511
[10, 29, 19]  vertice inicial = 482514
[1, 29, 4]  vertice inicial = 

[633, 613, 612]  vertice inicial = 485853
[635, 614, 613]  vertice inicial = 485856
[634, 615, 614]  vertice inicial = 485859
[632, 616, 615]  vertice inicial = 485862
[630, 617, 616]  vertice inicial = 485865
[631, 618, 617]  vertice inicial = 485868
[629, 620, 618]  vertice inicial = 485871
[642, 622, 620]  vertice inicial = 485874
[643, 622, 627]  vertice inicial = 485877
[626, 621, 643]  vertice inicial = 485880
[639, 623, 626]  vertice inicial = 485883
[640, 624, 639]  vertice inicial = 485886
[248, 252, 249]  vertice inicial = 485889
[251, 254, 249]  vertice inicial = 485892
[251, 255, 253]  vertice inicial = 485895
[255, 258, 256]  vertice inicial = 485898
[257, 260, 256]  vertice inicial = 485901
[257, 262, 259]  vertice inicial = 485904
[261, 264, 259]  vertice inicial = 485907
[261, 265, 263]  vertice inicial = 485910
[265, 268, 266]  vertice inicial = 485913
[267, 270, 266]  vertice inicial = 485916
[269, 272, 270]  vertice inicial = 485919
[271, 274, 272]  vertice inicial =

[475, 493, 494]  vertice inicial = 486519
[476, 494, 495]  vertice inicial = 486522
[477, 495, 496]  vertice inicial = 486525
[478, 496, 497]  vertice inicial = 486528
[479, 497, 498]  vertice inicial = 486531
[480, 498, 481]  vertice inicial = 486534
[481, 516, 499]  vertice inicial = 486537
[482, 499, 500]  vertice inicial = 486540
[483, 500, 501]  vertice inicial = 486543
[501, 502, 485]  vertice inicial = 486546
[502, 503, 486]  vertice inicial = 486549
[503, 504, 487]  vertice inicial = 486552
[504, 505, 489]  vertice inicial = 486555
[505, 506, 490]  vertice inicial = 486558
[506, 508, 507]  vertice inicial = 486561
[508, 509, 491]  vertice inicial = 486564
[509, 510, 492]  vertice inicial = 486567
[510, 528, 493]  vertice inicial = 486570
[493, 528, 511]  vertice inicial = 486573
[494, 511, 512]  vertice inicial = 486576
[495, 512, 513]  vertice inicial = 486579
[496, 513, 514]  vertice inicial = 486582
[497, 514, 515]  vertice inicial = 486585
[498, 515, 516]  vertice inicial =

[1289, 1280, 1290]  vertice inicial = 488949
[1290, 1281, 1291]  vertice inicial = 488952
[1291, 1282, 1292]  vertice inicial = 488955
[1292, 1283, 1293]  vertice inicial = 488958
[1294, 1293, 1284]  vertice inicial = 488961
[1295, 1284, 1285]  vertice inicial = 488964
[1296, 1285, 1286]  vertice inicial = 488967
[1297, 1286, 1287]  vertice inicial = 488970
[1298, 1287, 1288]  vertice inicial = 488973
[1299, 1288, 1289]  vertice inicial = 488976
[1299, 1290, 1300]  vertice inicial = 488979
[1300, 1291, 1301]  vertice inicial = 488982
[1301, 1292, 1302]  vertice inicial = 488985
[1302, 1293, 1303]  vertice inicial = 488988
[1304, 1303, 1294]  vertice inicial = 488991
[1305, 1294, 1295]  vertice inicial = 488994
[1306, 1295, 1296]  vertice inicial = 488997
[1307, 1296, 1297]  vertice inicial = 489000
[1308, 1297, 1298]  vertice inicial = 489003
[1309, 1298, 1299]  vertice inicial = 489006
[1309, 1300, 1310]  vertice inicial = 489009
[1310, 1301, 1311]  vertice inicial = 489012
[1311, 130

[2254, 2252, 2262]  vertice inicial = 491853
[2263, 2254, 2264]  vertice inicial = 491856
[2265, 2253, 2263]  vertice inicial = 491859
[2266, 2255, 2265]  vertice inicial = 491862
[2267, 2256, 2266]  vertice inicial = 491865
[2258, 2267, 2268]  vertice inicial = 491868
[2259, 2268, 2269]  vertice inicial = 491871
[2260, 2269, 2270]  vertice inicial = 491874
[2261, 2270, 2271]  vertice inicial = 491877
[2272, 2261, 2271]  vertice inicial = 491880
[2264, 2262, 2272]  vertice inicial = 491883
[2273, 2264, 2274]  vertice inicial = 491886
[2275, 2263, 2273]  vertice inicial = 491889
[2276, 2265, 2275]  vertice inicial = 491892
[2277, 2266, 2276]  vertice inicial = 491895
[2268, 2277, 2278]  vertice inicial = 491898
[2269, 2278, 2279]  vertice inicial = 491901
[2270, 2279, 2280]  vertice inicial = 491904
[2271, 2280, 2281]  vertice inicial = 491907
[2282, 2271, 2281]  vertice inicial = 491910
[2274, 2272, 2282]  vertice inicial = 491913
[2283, 2274, 2284]  vertice inicial = 491916
[2285, 227

...

[3622, 3621, 3631]  vertice inicial = 511704
[3613, 3622, 3632]  vertice inicial = 511707
[3624, 3623, 3633]  vertice inicial = 511710
[3635, 3625, 3624]  vertice inicial = 511713
[3636, 3626, 3625]  vertice inicial = 511716
[3637, 3627, 3626]  vertice inicial = 511719
[3638, 3628, 3627]  vertice inicial = 511722
[3639, 3629, 3628]  vertice inicial = 511725
[3640, 3630, 3629]  vertice inicial = 511728
[3631, 3630, 3640]  vertice inicial = 511731
[3632, 3631, 3641]  vertice inicial = 511734
[3623, 3632, 3642]  vertice inicial = 511737
[3634, 3633, 3643]  vertice inicial = 511740
[3645, 3635, 3634]  vertice inicial = 511743
[3646, 3636, 3635]  vertice inicial = 511746
[3647, 3637, 3636]  vertice inicial = 511749
[3648, 3638, 3637]  vertice inicial = 511752
[3649, 3639, 3638]  vertice inicial = 511755
[3650, 3640, 3639]  vertice inicial = 511758
[3641, 3640, 3650]  vertice inicial = 511761
[3642, 3641, 3651]  vertice inicial = 511764
[3633, 3642, 3652]  vertice inicial = 511767
[3644, 364

[3834, 3833, 3843]  vertice inicial = 512340
[3835, 3834, 3844]  vertice inicial = 512343
[3846, 3836, 3835]  vertice inicial = 512346
[3847, 3837, 3836]  vertice inicial = 512349
[3848, 3838, 3837]  vertice inicial = 512352
[3849, 3839, 3838]  vertice inicial = 512355
[3850, 3840, 3839]  vertice inicial = 512358
[3841, 3840, 3850]  vertice inicial = 512361
[3842, 3841, 3851]  vertice inicial = 512364
[3833, 3842, 3852]  vertice inicial = 512367
[3844, 3843, 3853]  vertice inicial = 512370
[3845, 3844, 3854]  vertice inicial = 512373
[3856, 3846, 3845]  vertice inicial = 512376
[3857, 3847, 3846]  vertice inicial = 512379
[3858, 3848, 3847]  vertice inicial = 512382
[3859, 3849, 3848]  vertice inicial = 512385
[3860, 3850, 3849]  vertice inicial = 512388
[3851, 3850, 3860]  vertice inicial = 512391
[3852, 3851, 3861]  vertice inicial = 512394
[3843, 3852, 3862]  vertice inicial = 512397
[3854, 3853, 3863]  vertice inicial = 512400
[3855, 3854, 3864]  vertice inicial = 512403
[3866, 385

[4278, 4268, 4267]  vertice inicial = 513642
[4279, 4269, 4268]  vertice inicial = 513645
[4280, 4270, 4269]  vertice inicial = 513648
[4281, 4271, 4270]  vertice inicial = 513651
[4272, 4271, 4281]  vertice inicial = 513654
[4263, 4272, 4282]  vertice inicial = 513657
[4274, 4273, 4283]  vertice inicial = 513660
[4275, 4274, 4284]  vertice inicial = 513663
[4276, 4275, 4285]  vertice inicial = 513666
[4287, 4277, 4276]  vertice inicial = 513669
[4288, 4278, 4277]  vertice inicial = 513672
[4289, 4279, 4278]  vertice inicial = 513675
[4290, 4280, 4279]  vertice inicial = 513678
[4291, 4281, 4280]  vertice inicial = 513681
[4282, 4281, 4291]  vertice inicial = 513684
[4273, 4282, 4292]  vertice inicial = 513687
[4284, 4283, 4293]  vertice inicial = 513690
[4285, 4284, 4294]  vertice inicial = 513693
[4286, 4285, 4295]  vertice inicial = 513696
[4297, 4287, 4286]  vertice inicial = 513699
[4298, 4288, 4287]  vertice inicial = 513702
[4299, 4289, 4288]  vertice inicial = 513705
[4300, 429

[4834, 4833, 4843]  vertice inicial = 515340
[4835, 4834, 4844]  vertice inicial = 515343
[4836, 4835, 4845]  vertice inicial = 515346
[4837, 4836, 4846]  vertice inicial = 515349
[4848, 4838, 4837]  vertice inicial = 515352
[4849, 4839, 4838]  vertice inicial = 515355
[4850, 4840, 4839]  vertice inicial = 515358
[4851, 4841, 4840]  vertice inicial = 515361
[4852, 4842, 4841]  vertice inicial = 515364
[4833, 4842, 4852]  vertice inicial = 515367
[4844, 4843, 4853]  vertice inicial = 515370
[4845, 4844, 4854]  vertice inicial = 515373
[4846, 4845, 4855]  vertice inicial = 515376
[4847, 4846, 4856]  vertice inicial = 515379
[4858, 4848, 4847]  vertice inicial = 515382
[4859, 4849, 4848]  vertice inicial = 515385
[4860, 4850, 4849]  vertice inicial = 515388
[4861, 4851, 4850]  vertice inicial = 515391
[4862, 4852, 4851]  vertice inicial = 515394
[4843, 4852, 4862]  vertice inicial = 515397
[4854, 4853, 4863]  vertice inicial = 515400
[4855, 4854, 4864]  vertice inicial = 515403
[4856, 485

[5667, 5666, 5676]  vertice inicial = 517839
[5668, 5667, 5677]  vertice inicial = 517842
[5669, 5668, 5678]  vertice inicial = 517845
[5680, 5670, 5669]  vertice inicial = 517848
[5681, 5671, 5670]  vertice inicial = 517851
[5682, 5672, 5671]  vertice inicial = 517854
[5674, 5664, 5672]  vertice inicial = 517857
[5683, 5673, 5674]  vertice inicial = 517860
[5675, 5673, 5683]  vertice inicial = 517863
[5676, 5675, 5685]  vertice inicial = 517866
[5677, 5676, 5686]  vertice inicial = 517869
[5678, 5677, 5687]  vertice inicial = 517872
[5679, 5678, 5688]  vertice inicial = 517875
[5690, 5680, 5679]  vertice inicial = 517878
[5691, 5681, 5680]  vertice inicial = 517881
[5692, 5682, 5681]  vertice inicial = 517884
[5684, 5674, 5682]  vertice inicial = 517887
[5693, 5683, 5684]  vertice inicial = 517890
[5685, 5683, 5693]  vertice inicial = 517893
[5686, 5685, 5695]  vertice inicial = 517896
[5687, 5686, 5696]  vertice inicial = 517899
[5688, 5687, 5697]  vertice inicial = 517902
[5689, 568

#### Mesa

In [21]:
modelo = load_model_from_file('cenario/mesa/mesa.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo mesa.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo mesa.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(15,'cenario/mesa/mesa.jpg')

Processando modelo mesa.obj. Vertice inicial: 518529
Processando modelo mesa.obj. Vertice final: 619713


#### Caminho

In [22]:
modelo = load_model_from_file('cenario/terreno/caminho.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo mesa.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo mesa.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(17,'cenario/terreno/pedra.jpg')

Processando modelo mesa.obj. Vertice inicial: 619713
Processando modelo mesa.obj. Vertice final: 619725


#### Ovni

In [23]:
modelo = load_model_from_file('cenario/ovni/ovni.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo ovni.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo ovni.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(18,'cenario/ovni/ovni.jpg')

Processando modelo ovni.obj. Vertice inicial: 619725
Processando modelo ovni.obj. Vertice final: 845709


#### Skybox (céu)

In [24]:
modelo = load_model_from_file('cenario/terreno/ceu.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo ceu1.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo ceu1.obj. Vertice final:',len(vertices_list))


load_texture_from_file(19,'cenario/terreno/1.jpg')
load_texture_from_file(20,'cenario/terreno/2.jpg')
load_texture_from_file(21,'cenario/terreno/3.jpg')
load_texture_from_file(22,'cenario/terreno/4.jpg')
load_texture_from_file(23,'cenario/terreno/5.jpg')

Processando modelo ceu1.obj. Vertice inicial: 845709
Processando modelo ceu1.obj. Vertice final: 845745


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar slots.

Nós agora vamos requisitar dois slots.
* Um para enviar coordenadas dos vértices.
* Outros para enviar coordenadas de texturas.

In [25]:
buffer = glGenBuffers(3)

###  Enviando coordenadas de vértices para a GPU

In [26]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

###  Enviando coordenadas de textura para a GPU

In [27]:
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)###  Enviando dados de Iluminação a GPU

###  Enviando dados de Iluminação a GPU

#### Dados de iluminação: vetores normais

In [28]:
normals = np.zeros(len(normals_list), [("position", np.float32, 3)]) # três coordenadas
normals['position'] = normals_list


# Upload coordenadas normals de cada vertice
glBindBuffer(GL_ARRAY_BUFFER, buffer[2])
glBufferData(GL_ARRAY_BUFFER, normals.nbytes, normals, GL_STATIC_DRAW)
stride = normals.strides[0]
offset = ctypes.c_void_p(0)
loc_normals_coord = glGetAttribLocation(program, "normals")
glEnableVertexAttribArray(loc_normals_coord)
glVertexAttribPointer(loc_normals_coord, 3, GL_FLOAT, False, stride, offset)

### Desenhando nossos modelos
* Cada modelo tem um Model para posicioná-los no mundo.
* É necessário saber qual a posição inicial e total de vértices de cada modelo
* É necessário indicar qual o ID da textura do modelo


In [29]:
def defineParametrosIluminacao(ka, kd, ks, ns):
    
    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel kd na GPU
    glUniform1f(loc_kd, kd) ### envia kd pra gpu    
    
    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks) ### envia ns pra gpu        
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu

In [30]:
def desenha_terreno():
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = -1.01; t_z = 0.0;
    
    # escala
    s_x = 500.0; s_y = 500.0; s_z = 500.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    
    defineParametrosIluminacao(luzAmb, 0.5, 0.8, 1)
    
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 1)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 0, 6) ## renderizando

In [31]:
def desenha_casa():
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = -1.0; t_z = 0.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    
    defineParametrosIluminacao(luzAmb, 0.5, 0.8, 1)
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 2)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 6, 2628-6) ## renderizando

In [32]:
def desenha_cama():
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = -5.0; t_y = -0.3; t_z = -0.5;
    
    # escala
    s_x = 1.3; s_y = 1.3; s_z = 1.3;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    defineParametrosIluminacao(luzAmb, 0.5, 0.8, 1)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 3)    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 2592, 6108-2592) ## renderizando
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 4)    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 6108, 6432-6108) ## renderizando
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 5)    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 6432, 6600-6432) ## renderizando

In [33]:
def desenha_fogao():
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = -1.0; t_z = 0.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    defineParametrosIluminacao(luzAmb, 0.5, 0.0, 0.1)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 6)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 6600, 215709-6600) ## renderizando

In [34]:
def desenha_gato(rotacao_inc):
    
    # rotacao
    angle = rotacao_inc;
    r_x = 0.0; r_y = -1.0; r_z = 0.0;
    
    # translacao
    t_x = 0.0; t_y = -1.0; t_z = 0.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    defineParametrosIluminacao(luzAmb, 0.5, 0.8, 1)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 7)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 215709, 427437-215709) ## renderizando

In [35]:
def desenha_cacto():
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = -1.0; t_z = 0.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    defineParametrosIluminacao(luzAmb, 0.5, 0.8, 1)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 8)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 427437, 482445-427437) ## renderizando

In [36]:
def desenha_poco():
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = -1.0; t_z = 0.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    defineParametrosIluminacao(luzAmb, 0.5, 0.8, 1)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 9)    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 482445, 482805-482445) ## renderizando
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 10)    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 482805, 484113-482805) ## renderizando
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 11)    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 484113, 484485-484113) ## renderizando
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 12)    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 484485, 484689-484485) ## renderizando
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 13)    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 484689, 487041-484689) ## renderizando
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 14)    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 487041, 518529-487041) ## renderizando

In [37]:
def desenha_mesa():
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = -1.0; t_z = 0.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    defineParametrosIluminacao(luzAmb, 0.5, 0.8, 1)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 15)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 518529, 619713-518529) ## renderizando

In [38]:
def desenha_caminho():
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = -1.00; t_z = 0.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    defineParametrosIluminacao(luzAmb, 0.5, 0.8, 1)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 17)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 619713, 619725-619713) ## renderizando

In [39]:
def desenha_ovni(t_x, t_y, t_z):
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = -1.0; r_z = 0.0;
    
    # translacao
    #t_x = 0.0; t_y = -1.00; t_z = 0.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    defineParametrosIluminacao(luzAmb, 0.5, 0.8, 1)
    

    
    loc_light_pos = glGetUniformLocation(program, "lightPos") # recuperando localizacao da variavel lightPos na GPU
    glUniform3f(loc_light_pos, t_x, t_y, t_z) ### posicao da fonte de luz
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 18)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 619725, 845709-619725) ## renderizando

# Processando modelo ovni.obj. Vertice inicial: 619725
# Processando modelo ovni.obj. Vertice final: 845709

In [40]:
def desenha_ceu():
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = -1.00; t_z = 15.0;
    
    # escala
    s_x = 500.0; s_y = 500.0; s_z = 500.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    
    defineParametrosIluminacao(luzAmb, 0.0, 0.0, 100000000000.0)
    
       
    #define id da textura do modelo ---- 1 ----
    glBindTexture(GL_TEXTURE_2D, 19)   
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 845709, 3) ## renderizando
    
    #define id da textura do modelo ---- 2 ----
    glBindTexture(GL_TEXTURE_2D, 20)   
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 845709+3, 3) ## renderizando

    
    #define id da textura do modelo ---- 3 ----
    glBindTexture(GL_TEXTURE_2D, 21)   
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 845709+6, 3) ## renderizando

    
    #define id da textura do modelo ---- 4 ----
    glBindTexture(GL_TEXTURE_2D, 22)   
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 845709+9, 3) ## renderizando

    
    #define id da textura do modelo ---- 5 ----
    glBindTexture(GL_TEXTURE_2D, 23)   
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 845709+12, 3) ## renderizando
    
    #define id da textura do modelo ---- 6 ----
    glBindTexture(GL_TEXTURE_2D, 23)   
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 845709+15, 3) ## renderizando
    
    #define id da textura do modelo ---- 1 ----
    glBindTexture(GL_TEXTURE_2D, 19)   
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 845709+18, 3) ## renderizando

    
    #define id da textura do modelo ---- 2 ----
    glBindTexture(GL_TEXTURE_2D, 20)   
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 845709+21, 3) ## renderizando

    
    #define id da textura do modelo ---- 3 ----
    glBindTexture(GL_TEXTURE_2D, 21)   
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 845709+24, 3) ## renderizando
    
    #define id da textura do modelo ---- 4 ----
    glBindTexture(GL_TEXTURE_2D, 22)   
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 845709+27, 3) ## renderizando
    
    #define id da textura do modelo ---- 5 ----
    glBindTexture(GL_TEXTURE_2D, 23)   
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 845709+30, 3) ## renderizando
    
    #define id da textura do modelo ---- 6 ----
    glBindTexture(GL_TEXTURE_2D, 23)   
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 845709+33, 3) ## renderizando


# Processando modelo ceu1.obj. Vertice inicial: 845709
# Processando modelo ceu1.obj. Vertice final: 845745

### Eventos para modificar a posição da câmera.

* Usei as teclas A, S, D e W para movimentação no espaço tridimensional
* Usei a posição do mouse para "direcionar" a câmera

In [41]:
cameraPos   = glm.vec3(0.0,  0.0,  1.0);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);
cameraTest  = glm.vec3(-1.0,  0.0,  0.0);


polygonal_mode = False
luzAmb = 0.8

def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode, ns_inc, cv, luzAmb
    
    cameraSpeed = 0.2      
        
        
    if key == 87 and (action==1 or action==2): # tecla W
        cameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed

    if key == 263 and (action==1 or action==2): # tecla <-
        cameraPos += glm.normalize(glm.cross(cameraTest, cameraFront)) * cameraSpeed
        
    if key == 262 and (action==1 or action==2): # tecla ->
        cameraPos -= glm.normalize(glm.cross(cameraTest, cameraFront)) * cameraSpeed
        
    if key == 265 and (action==1 or action==2): # tecla up
        ns_inc = ns_inc * 2
        
    if key == 264 and (action==1 or action==2): # tecla down
        ns_inc = ns_inc / 2
    

    if key == 77 and (action==1 or action==2) and cv < 115.0: # tecla up
        cv += 2.5
        
    if key == 78 and (action==1 or action==2) and cv > 30.0: # tecla down
        cv -= 2.5
        
    if key == 80 and (action==1 or action==2) and cv < 115.0: # tecla up
        luzAmb += 0.1
        
    if key == 85 and (action==1 or action==2) and cv > 30.0: # tecla down
        luzAmb -= 0.1
        
    
#     if key == 80 and action==1 and polygonal_mode==True:
#         polygonal_mode=False
#     else:
#         if key == 80 and action==1 and polygonal_mode==False:
#             polygonal_mode=True
            
        
            
    # delimitando a exploração do cenário
    if cameraPos.x > 50: cameraPos.x = 50
    if cameraPos.x < -50: cameraPos.x = -50
    if cameraPos.y > 50: cameraPos.y = 50
    if cameraPos.y < -0.5: cameraPos.y = -0.5
    if cameraPos.z > 50: cameraPos.z = 50
    if cameraPos.z < -50: cameraPos.z = -50  
    

cv = 45.0
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)


    
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)

### Matrizes Model, View e Projection

Teremos uma aula específica para entender o seu funcionamento.

In [42]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    
    angle = math.radians(angle)
    
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade

    
    # aplicando translacao
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))    
    
    # aplicando rotacao
    matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))
    
    # aplicando escala
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    matrix_transform = np.array(matrix_transform).T # pegando a transposta da matriz (glm trabalha com ela invertida)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(cv), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Nesse momento, nós exibimos a janela!


In [43]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)
glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

In [44]:
glEnable(GL_DEPTH_TEST) ### importante para 3D

rotacao_inc = 0
ang = 0

while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    if polygonal_mode==True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode==False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
    
    ang += 0.01
    rotacao_inc += 0.7
    
    #desenha_skybox()
    desenha_terreno()
    desenha_caminho()
    desenha_casa()
    desenha_cama()
    desenha_fogao()
    desenha_gato(rotacao_inc)
    desenha_cacto()
    desenha_poco()
    desenha_ovni(math.cos(ang)*40, 10.0, math.sin(ang)*40)
    desenha_mesa()
    desenha_ceu()
    
    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_FALSE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_FALSE, mat_projection)
    
    glfw.swap_buffers(window)

glfw.terminate()